In [64]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import json
import time
import os

from src.neuron_heads import head_attribution_over_all_data
from src.datahandlers import ActivatingDataset
from src.utils import tuple_str_to_tuple
from src.neuron_explain import generate_classify_prompt_dict

In [48]:
# filename = "2023-10-13_13-51-37_gpt2-small_test"
# filename = "2023-10-13_14-02-06_gpt2-medium_test"
filename = "2023-10-13_14-17-42_gpt2-large_test"

with open(f'../experiment_data/4_head_attributions/{filename}.json') as f:
    head_attributions = json.load(f)

trimmed_texts_filename = head_attributions['prior_filename']
with open(f'../experiment_data/3_trimmed_texts/{trimmed_texts_filename}.json') as f:
    trimmed_texts = json.load(f)

max_activating_filename = trimmed_texts['prior_filename']
with open(f'../experiment_data/2_max_activating_texts/{max_activating_filename}.json') as f:
    max_activating = json.load(f)

neuron_filename = max_activating['prior_filename']
with open(f'../experiment_data/1_next_token_neurons/{neuron_filename}.json') as f:
    neurons_data = json.load(f)

neurons = [tuple_str_to_tuple(neuron_str) for neuron_str in head_attributions['head_attributions'].keys()]

neuron_to_token = {tuple_str_to_tuple(neuron_str): token_data['token'] for neuron_str, token_data in neurons_data['neurons'].items()}

# Parameters
parameters = head_attributions['parameters']
model_name = parameters['model_name']

In [49]:
# Load explanations

import json
# Load the jsonl
with open(f'../experiment_data/5_head_explanations/2024-02-15_01-59-23_pythia-1.4b_train_results.jsonl', 'r') as f:
    lines = f.readlines()
    lines = [json.loads(line) for line in lines]

x = lines[0]
# print(x[0]['messages'])
exp_prompt_to_exp = {x[0]['messages'][0]['content']: x[1]['choices'][0]['message']['content'] for x in lines}

with open('../experiment_data/5_head_explanations/2024-02-15_01-59-23_pythia-1.4b_train_prompts_dict.json') as f:
    nh_to_exp_prompt = json.load(f)

nh_to_exp_prompt = {tuple_str_to_tuple(k): v for k, v in nh_to_exp_prompt.items()}

nh_to_exp = {nh: exp_prompt_to_exp[exp_prompt] for nh, exp_prompt in nh_to_exp_prompt.items()}

In [50]:
import pickle

with open('../experiment_data/text_list_dict_test.pkl', 'rb') as f:
    dataset_text_dict = pickle.load(f)

dataset_text_list = [x['text'] for x in dataset_text_dict]
dataset = dataset_text_dict

In [51]:
neuron_to_prompts = {tuple_str_to_tuple(neuron): 
                     list(head_attribution.keys()) 
                     for neuron, head_attribution in head_attributions['head_attributions'].items()}

In [52]:
gpt_4_prompts_dict = generate_classify_prompt_dict(
    nh_to_exp = nh_to_exp,
    neuron_to_prompts=neuron_to_prompts,
    neuron_to_token=neuron_to_token,
    max_per_nh=10,
)

In [53]:
gpt_4_prompts = []
for nh, exp in gpt_4_prompts_dict.items():
    gpt_4_prompts += list(exp.keys())

len(gpt_4_prompts)

13460

In [42]:
len(gpt_4_prompts)

2660

In [54]:
jobs = [
            {"model":"gpt-4-1106-preview",
            "messages":[{"role": "user", "content": gpt_4_prompt}],
            "max_tokens":1, 
        } for gpt_4_prompt in gpt_4_prompts]

filepath = f"../experiment_data/6_explanation_eval/{filename}_mid_aclcn.jsonl"
if os.path.isfile(filepath):
    raise Exception("File already exists!")

with open(filepath, "w") as f:
    for job in jobs:
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [55]:
output = {
    'parameters': parameters,
    'prior_filename': filename,
    'classify_filepath': filepath,
    'classify_prompts': {str(k):v for k,v in gpt_4_prompts_dict.items()}
}

# Save json to ../experiment_data/2_max_activating_texts
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime(int(time.time())))
new_filename = f"{timestamp}_{model_name}_test_mid_aclcn.json"

with open(f'../experiment_data/6_explanation_eval/{new_filename}', 'w') as f:
    json.dump(output, f)

In [56]:
prompt_token_cost = 0.01/1000

import tiktoken
encoding = tiktoken.encoding_for_model('gpt-4')

# Count the number of tokens in `gpt_4_prompts`
prompt_tokens = 0
for gpt_4_prompt in gpt_4_prompts:
    prompt_tokens += len(encoding.encode(gpt_4_prompt))



print(prompt_tokens)
print(prompt_tokens * prompt_token_cost)

2318858
23.18858
